# Readme
1. Define point features for plotting: upland forest coordinates, and polyogon
2. Filtering functions for Landsat images
3. Filtered dataset function
4. Download NDVI image to google drive
5. Map features

The final pr

# Import Libraries and Initiate Project

In [40]:
import ee
import geemap
import csv
import geopandas as gpd
from shapely import wkt
import pandas as pd
from tqdm import tqdm  # Optional: for progress bar 
from geopandas import GeoSeries
from shapely.geometry import Point


ee.Authenticate()
ee.Initialize(project='ee-typrins')

Loading Data

## 1. Define features 

In [41]:
abrupt_file = '/home/jovyan/sara_typrin/ndvi_landsat/data/forest_pathway/abrupt.csv'
gradual_file = '/home/jovyan/sara_typrin/ndvi_landsat/data/forest_pathway/gradual.csv'
recovery_file = '/home/jovyan/sara_typrin/ndvi_landsat/data/forest_pathway/recovery.csv'
stable_file = '/home/jovyan/sara_typrin/ndvi_landsat/data/forest_pathway/stable.csv'


# Read each CSV into a list of DataFrames
abrupt = pd.read_csv(abrupt_file) 
gradual = pd.read_csv(gradual_file) 
recovery = pd.read_csv(recovery_file)
stable = pd.read_csv(stable_file)

forest_pathway_list = [abrupt, gradual, recovery, stable]

coords_df = '/home/jovyan/sara_typrin/landcover/data/coords_df.csv'
coords_df = pd.read_csv(coords_df)  # coords_df is a file path (e.g., 'data/coords.csv')

In [42]:

# Create Point geometry from longitude and latitude columns
geometry = [Point(xy) for xy in zip(coords_df['lon'], coords_df['lat'])]

# Convert to GeoDataFrame with point geometry
coords_gdf = gpd.GeoDataFrame(coords_df, geometry=geometry)

# Set coordinate reference system to WGS84 (lat/lon)
coords_gdf.set_crs(epsg=4326, inplace=True)

,lat,lon,geometry
0,38.673966,-76.340737,POINT (-76.34074 38.67397)
1,38.673699,-76.340387,POINT (-76.34039 38.6737)
2,38.673163,-76.340032,POINT (-76.34003 38.67316)
3,38.674662,-76.202116,POINT (-76.20212 38.67466)
4,38.674665,-76.201771,POINT (-76.20177 38.67467)
...,...,...,...
583990,38.199449,-75.690616,POINT (-75.69062 38.19945)
583991,38.199455,-75.689588,POINT (-75.68959 38.19945)
583992,38.199461,-75.688560,POINT (-75.68856 38.19946)
583993,38.199463,-75.688218,POINT (-75.68822 38.19946)


In [43]:
# # Extract WKT strings from the 'point' column
# wkt_list = coords.point.tolist()

# # Convert WKT strings to Shapely geometries
# geoms = [wkt.loads(w) for w in wkt_list]

# # Create GeoDataFrame
# gdf = gpd.GeoDataFrame(coords_df.drop(columns=['point']), geometry=geoms, crs="EPSG:4326")

# # Export as GeoJSON
# gdf.to_file("points.geojson", driver="GeoJSON")
# print("Saved as points.geojson ✅")

### Add geojason points to Earth Engine Feature Collection.

In [44]:
# #put gdf into geometry form
# ee_points = [
#     ee.Feature(ee.Geometry.Point(p.x, p.y), {
#         "type_value": row.type_value,
#         "type": row.type
#     })
#     for p, row in zip(gdf.geometry, gdf.itertuples(index=False))
# ]
# points = ee.FeatureCollection(ee_points)

In [45]:
polygon = ee.Geometry.Polygon([
    [-76.35156992998947, 38.19934796579719],
    [-75.68415049639572, 38.19934796579719],
    [-75.68415049639572, 38.67479854331683],
    [-76.35156992998947, 38.67479854331683],
    [-76.35156992998947, 38.19934796579719], 
])

## 2. Filtering Functions

In [46]:
def clp(image):
    '''Clips a single Image to a region of interest'''
    return image.clip(polygon)

In [47]:
# Applies scaling factors.
def apply_scale_factors(image):
  optical_bands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
  thermal_bands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
  return image.addBands(optical_bands, None, True).addBands(
      thermal_bands, None, True
  )

In [48]:
def maskS2clouds(image):
    qa = image.select('QA_PIXEL')

    # Bits 10 and 11 are clouds and cirrus, respectively.
    cloudBitMask = 1 << 3
    cirrusBitMask = 1 << 4

    # Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloudBitMask).eq(0) \
        .And(qa.bitwiseAnd(cirrusBitMask).eq(0))

    return image.updateMask(mask) \
        .copyProperties(image, ['system:time_start'])  # this guy is important!

In [49]:
# Make an NDVI band
def addNDVI(image):
    ndvi = image.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDVI')
    #ndvi=np.where(
    #(nir+red)==0., 
    #0, 
    #(nir-red)/(nir+red))
    return image.addBands(ndvi).copyProperties(image, ['system:time_start'])

In [50]:
# Function to sample NDVI at all points for each image
def sample_image(image):
    # Sample NDVI at points
    samples = image.select('NDVI').sampleRegions(
        points,
        scale=30,
        geometries=True  # Keep the geometry in output
    )
    # Add image date property to each sample
    date_str = image.date().format('YYYY-MM-dd')
    samples = samples.map(lambda f: f.set('date', date_str))
    return samples

# 3. Filtering Dataset

In [2]:
filtered_dataset = (
    ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")  
                #   // Sentinel 2 harmonized data only available for certain time
                .filter(ee.Filter.calendarRange(2024, 2024, 'year')) #choose range for desire image to download
                #   // Filter by month. Be mindful of snow! 
                .filter(ee.Filter.calendarRange(6,8,'month')) #choose 6-8 for growing season
                #.filterDate('2014-02-01','2020-05-30')
                .filterBounds(polygon)
                .map(clp)
                .map(apply_scale_factors)
                .filterMetadata('CLOUD_COVER', 'less_than', 20)
                .map(maskS2clouds)
                .map(addNDVI)
                .select('NDVI')
)

NameError: name 'ee' is not defined

In [52]:
def get_year(image):
    date = ee.Date(image.get('system:time_start'))
    year = date.get('year')
    return image.set('year', year)

# Add 'year' metadata to each image
dataset_with_years = filtered_dataset.map(get_year)

# Group and count images by year
years = (dataset_with_years.aggregate_histogram('year').getInfo())

# Print results
for year, count in sorted(years.items()):
    print(f"{year}: {count} images")

2024: 2 images


In [53]:

# Count the number of images
image_count = filtered_dataset.size().getInfo()

print(f"Number of images: {image_count}")

Number of images: 2


Define dataset as image collection with filters applied

In [54]:
image = filtered_dataset.max()

# 4. Download NDVI Image 
#### For each image downloaded, make sure the date range in the dataset function matches the year you want the image.

In [55]:
# Export the image, specifying the CRS, transform, and region.
task = ee.batch.Export.image.toDrive(
    image=image,
    description='ndvi_landsat_2024',
    region=polygon,
    folder='ndvi_landsat', 
    scale=30,
    maxPixels=1e13
)
task.start()

In [3]:
task.status() #Refresh this line to see current status

NameError: name 'task' is not defined

Global Variables Defined

Export NDVI Images

In [1]:
print(filtered_dataset.first().bandNames().getInfo())

NameError: name 'filtered_dataset' is not defined

# 5. Map

In [ ]:
visParams = {'bands': ['SR_B2', 'SR_B4', 'SR_B3'], 'min': 1, 'max': 65455} # RGB
image_visParams = {
    'bands': ['SR_B4', 'SR_B3', 'SR_B2'],  # RGB
    'min': 0,
    'max': 0.3,
}
#point_style = aoi.style(
#    color='071aff',    # blue color
#    width=3,           # point outline width
 #   fillColor='071aff' # fill color
#)

# Create a FeatureCollection from the geometry
fc = ee.FeatureCollection([ee.Feature(polygon)])

point_style = points.style(
    color='071aff',    # blue color
    width=3,           # point outline width
    fillColor='071aff' # fill color
)

# Style the FeatureCollection with transparent fill and red outline
style_params = {
    'fillColor': '00000000',  # Transparent fill
    'color': 'FF0000',        # Red outline
    'width': 2,               # Line width
    'lineType': 'solid'       # Solid line
}
styled_image = fc.style(**style_params)

#'pallete'
m = geemap.Map(center= [37.8, -76.2], zoom=5)

# Add styled point layer to map
m.addLayer(point_style, {}, 'Points')
#m.addLayer(dataset, visParams, 'true color composite')
#m.addLayer(image1, image_visParams, 'Image')
#m
#m.addLayer(styled_image, {}, 'Styled Polygon')
#m.addLayer(image2024, {}, 'ndvi2024')
m